# EXPLORATION _ 9 프로젝트 - Movielens 영화 추천 실습

### 추천시스템의 MNIST라고 부를만한 Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
-별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [ ]:
! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

## 1. 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [2]:

import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# timstamp는 사용하지 않으므로 제거
# count 컬럼을 앞으로
using_cols = ['user_id', 'count', 'movie_id']
ratings = ratings[using_cols]
ratings.tail(10)

,user_id,count,movie_id
1000198,6040,3,2021
1000199,6040,5,2022
1000200,6040,5,2028
1000201,6040,4,1080
1000202,6040,4,1089
1000203,6040,3,1090
1000205,6040,5,1094
1000206,6040,5,562
1000207,6040,4,1096
1000208,6040,4,1097


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 데이터 분석

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [8]:
# ratings와 movies를 합쳐서 영화제목 출력
total = pd.merge(ratings, movies)
print(len(total))

836478


In [9]:
ratings.nunique()

user_id     6039
count          3
movie_id    3628
dtype: int64

In [10]:
ratings['movie_id'].nunique()

3628

In [11]:
# 인기 많은 영화 Top-30
movie_count = total.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [12]:
# 영화 제목 찾기
movie_temp = movies.loc[movies['genre'].str.contains('Comedy')].copy()
movie_temp['title'].unique()

array(['Toy Story (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', ..., 'Bamboozled (2000)',
       'Bootmen (2000)', 'Meet the Parents (2000)'], dtype=object)

In [13]:
# 내가 좋아 하는 영와 리스트 만들기
my_favorite = ['Toy Story (1995)', 'Grumpier Old Men (1995)','Waiting to Exhale (1995)', 'Bootmen (2000)', 'Meet the Parents (2000)']
my_favorite_id = movies[movies['title'].isin(my_favorite)]
my_favorite_id

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
3874,3944,Bootmen (2000),Comedy|Drama
3878,3948,Meet the Parents (2000),Comedy


In [14]:
# 'jee_id'이라는 user_id가 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['jee_id']*5, 'count':['5']*5, 'movie_id': my_favorite_id['movie_id']})

if not ratings.isin({'user_id':['jee_id']})['user_id'].any():  # user_id에 'jee_id'이라는 데이터가 없다면
    ratings= ratings.append(my_movielist)                           # 위에 임의로 만든 my_movielist 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,count,movie_id
1000203,6040,3,1090
1000205,6040,5,1094
1000206,6040,5,562
1000207,6040,4,1096
1000208,6040,4,1097
0,jee_id,5,1
2,jee_id,5,3
3,jee_id,5,4
3874,jee_id,5,3944
3878,jee_id,5,3948


## 4. CSR matrix를 직접 만들기

In [15]:
total= pd.merge(ratings, movies).copy()
total.tail()

,user_id,count,movie_id,title,genre
836478,5851,5,3607,One Little Indian (1973),Comedy|Drama|Western
836479,5854,4,3026,Slaughterhouse (1987),Horror
836480,5854,3,690,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,4,2909,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,5,1360,Identification of a Woman (Identificazione di ...,Drama


In [19]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = total['user_id'].unique()
movie_unique = total['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
#print(user_to_idx['jee_id'])     
#print(movie_to_idx['Bootmen (2000)'])

In [24]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = total['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(total):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    total['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = total['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(total):
    print('title column indexing OK!!')
    total['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

total

user_id column indexing Fail!!
title column indexing Fail!!


,user_id,count,movie_id,title,genre
0,0,5,1193,0,Drama
1,1,5,1193,0,Drama
2,2,4,1193,0,Drama
3,3,4,1193,0,Drama
4,4,5,1193,0,Drama
...,...,...,...,...,...
836478,1621,5,3607,3623,Comedy|Drama|Western
836479,3481,4,3026,3624,Horror
836480,3481,3,690,3625,Romance
836481,4159,4,2909,3626,Documentary


In [25]:
num_user = total['user_id'].nunique()
num_movie = total['title'].nunique()

In [26]:
total['count']

0         5
1         5
2         4
3         4
4         5
         ..
836478    5
836479    4
836480    3
836481    4
836482    5
Name: count, Length: 836483, dtype: object

In [27]:
# CSR Matrix 생성
from scipy.sparse import csr_matrix

num_user = total['user_id'].nunique()
num_movie = total['title'].nunique()

csr_data = csr_matrix((total['count'], (total.user_id, total.title)), shape= (num_user, num_movie))
csr_data

TypeError: no supported conversion for types: (dtype('O'),)

# 오류해결이 되지 않아...


# 진행이 불가합니다.

# 다시 처음부터 차근히 진행하고 다시 제출하겠습니다.

## 5.als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [1]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

NameError: name 'os' is not defined